In [5]:
import re
import jieba
import pandas as pd

typetxt = open('stopwords.txt')
texts = ['\u3000','\n',' ']

for word in typetxt:
    word = word.strip()
    texts.append(word)
# 读入数据
filename = pd.read_csv('data.csv', header = None)
filename.head()

,0,1
0,霉霉或将献唱《芭比》电影原声带 官方暂未确认,娱乐
1,许光汉回应加盟《中餐厅》：可以期待一下,娱乐
2,小S说自己有容貌焦虑：在餐厅吃饭会不停照镜子,娱乐
3,龟梨和也出演三池崇史新片《怪物樵夫》,娱乐
4,福建省泉州市丰泽区影视文旅产业推介会举办,娱乐


In [6]:
len(filename)

10000

In [7]:
labels=[]
corpus=[]
cont = []
for i in range(len(filename)):
    cont.append(filename.loc[i,0])
    doc = str(filename.loc[i,0])
    labels.append(filename.loc[i,1])
    data = jieba.cut(doc) # 文本分词
    data_adj=''
    delete_word=[]
    for item in data:
        if item not in texts: # 停用词过滤
            value = re.compile(r'^[\u4e00-\u9fa5]{2,}$') # 只匹配中文2字词以上
            if value.match(item):
                data_adj+=item+' '
        else:
            delete_word.append(item)
    corpus.append(data_adj) # 语料库建立完成
print(corpus)

['霉霉 芭比 电影 原声带 官方 暂未 确认 ', '许光汉 回应 加盟 中餐厅 期待 ', '容貌 焦虑 餐厅 吃饭 不停 照镜子 ', '出演 三池 崇史 新片 怪物 樵夫 ', '福建省 泉州市 丰泽区 影视 文旅 产业 推介会 举办 ', '音节 端午 假期 三亚 首秀 出演 阵容 公布 ', '新浪 观影 疯狂 元素 亲子 专场 免费 抢票 ', '新浪 观影 变形金刚 勇士 崛起 免费 抢票 ', '影迷 蜘蛛侠 纵横 宇宙 画风 炫酷 制作 精良 ', '高尔宣 北美 巡演 多伦多 官宣 开唱 ', '苏打 举办 限定 免费 演唱会 惊喜 巡演 ', '役所 广司 戛纳 影帝 凯旋 出席 活动 掌声 ', '新浪 观影 蜘蛛侠 纵横 宇宙 北京 免费 抢票 ', '郑钧 全国 巡回演唱 北京站 预售 开启 ', '影迷 大雄 天空 理想 温暖 感动 ', '志愿军 雄兵 出击 新浪潮 论坛 还原 历史 细节 ', '惊悚 剧集 母狮 发布 剧照 佐伊 索尔达 主演 ', '人生路不熟 上映 满月 范丞丞 发文 分享 感受 ', '苏炳添 照官 宣得子 暖心 配文 守护 妈妈 ', '真实 震撼 感动 并存 长空 之王 累计 票房 ', '深圳 声乐 公开课 上线 助力 中国 声乐 发展 ', '电视剧 公诉 定档 迪丽 热巴 大为 主演 ', '曝李仙姬 娱乐 代表 勾结 合力 侵占 公司 财产 ', '松井 玲奈 近藤 晃央 结婚 迪士尼 婚礼 ', '谢楠 节目 中谈 吴京 拍戏 最长 一年 回家 ', '导演 沙碧红 新片 现身 戛纳 影片 选材 真实 故事 ', '杨洋 沙溢 两位 艺校 华表奖 现场 互加 好友 ', '张馨予 发文 反对 虐待 动物 这是 可怕 蝴蝶效应 ', '星耀 曲江 演唱会 阵容 公布 预售 开启 ', '刘宪华 回应 当初 退出 向往 生活 压力 ', '花月 杀手 导演 马丁 时间 不多 ', '创造 宇宙 音乐节 范丞丞方 预付 门票 ', '西班牙 电影展 卢米 影城 开幕 ', '新浪 观影 天空 理想 免费 抢票 ', '电影 防线 启动 仪式 在京举行 ', '艺术 电影展 华夏 主题 展映 ', '刘诗 何超莲 合照 漂亮 姐姐 同框 赏心悦目 ', '杨洋 上海 出发 参加 华表奖 典礼 ', '

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
vectorizer=CountVectorizer() # 该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
vt = vectorizer.fit_transform(corpus)

In [9]:
transformer=TfidfTransformer() # 该类会统计每个词语的tf-idf权值
tfidf=transformer.fit_transform(vt) # 第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵
weight=tfidf.toarray() # 将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重

In [10]:
word=vectorizer.get_feature_names_out() # 获取词袋模型中的所有词
for j in range(len(word)):
    if weight[1][j]!=0:
        print(word[j], weight[1][j])

中餐厅 0.5411084546665599
加盟 0.3673032180604896
回应 0.29024141826701744
期待 0.410625659624787
许光汉 0.5651871133269359


In [11]:
from sklearn.cluster import KMeans

def Kmeans(weight,clusters,correct):
    mykms=KMeans(n_clusters=clusters)
    # 进行聚类
    y=mykms.fit_predict(weight)
    result=[]
    # 根据预测结果给文本添加对应标签
    for i in range(0,clusters):
        label_i = []
        cont_i = []
        yule = 0
        junshi = 0
        keji = 0
        caijing = 0
        for j in range(0,len(y)):
            # 简单的数值比较判断文本所属类别
            if y[j]==i:
                label_i.append(labels[j])
                cont_i.append(cont[j])
                type=labels[j][0:2]
                if(type=='娱乐'):
                    yule+=1
                elif(type=='军事'):
                    junshi+=1
                elif(type=='科技'):
                    keji+=1
                elif(type=='财经'):
                    caijing+=1
        max=junshi
        type='军事'
        if(yule>junshi):
            max=yule
            type='娱乐'
        if(max<keji):
            max=keji
            type='科技'
        if(max<caijing):
            max=caijing
            type='财经'
        correct[0]+=max
        result.append('类别'+'('+type+')'+':'+str(cont_i))
    return result

In [12]:
clusters=4 # 分4类
correct=[0] # 正确量
result=Kmeans(weight,clusters,correct)

In [13]:
size = 10000
def output(result,clusters):
    for i in range(0,clusters):
        print(result[i])
    print('本次分类总样本数目为:'+str(size)+' 其中正确分类数目为:'+str(correct[0])+' 正确率为：'+str(correct[0]/size))

In [14]:
# 查看分类结果
output(result,clusters)

类别(军事):['霉霉或将献唱《芭比》电影原声带 官方暂未确认', '许光汉回应加盟《中餐厅》：可以期待一下', '小S说自己有容貌焦虑：在餐厅吃饭会不停照镜子', '龟梨和也出演三池崇史新片《怪物樵夫》', '福建省泉州市丰泽区影视文旅产业推介会举办', 'S2O电音节端午假期将在三亚首秀 出演阵容公布', '影迷评《蜘蛛侠：纵横宇宙》画风炫酷 制作精良', 'OSN高尔宣北美巡演多伦多站官宣 9月1日将开唱', '苏打绿举办限定免费演唱会 宣布将有惊喜巡演', '役所广司获得戛纳影帝凯旋 出席活动获掌声', '2023郑钧全国巡回演唱会北京站5月31日预售开启', '影迷评《哆啦A梦：大雄与天空的理想乡》温暖感动', '《志愿军：雄兵出击》新浪潮论坛 还原历史细节', '惊悚剧集《母狮》发布剧照 佐伊·索尔达娜主演', '《人生路不熟》上映满月 范丞丞发文分享感受', '苏炳添晒照官宣得子 暖心配文“一起守护妈妈”', '真实震撼感动并存 《长空之王》累计票房破8亿', '第四届深圳声乐季公开课上线 助力中国声乐发展', '电视剧《公诉》定档5月29日 迪丽热巴佟大为主演', 'SKE48松井玲奈将与近藤晃央结婚 在迪士尼办婚礼', '谢楠节目中谈吴京 称他为了拍戏最长一年半没回家', '导演沙碧红携新片现身戛纳 影片选材真实故事', '杨洋和沙溢两位军艺校草在华表奖现场互加好友', '张馨予发文反对虐待动物：这是可怕的蝴蝶效应', '2023“星耀曲江”演唱会阵容公布 预售即将开启', '刘宪华回应当初退出《向往的生活》：压力太大', '《花月杀手》7年才完成 导演马丁:我的时间不多了', '创造宇宙音乐节将举行 范丞丞方预付2000张门票', '2023西班牙电影展即将在卢米埃影城开幕', '电影《最后的防线》启动仪式在京举行', '艺术电影展·2023春华夏繁主题展映', '刘诗诗与何超莲合照 漂亮姐姐同框赏心悦目', '杨洋上海出发 将参加华表奖典礼', '小鬼王琳凯生日演唱会启动 巡演首站登陆深圳', '黄渤倪妮新戏在港拍摄发生意外 伤者正在诊断治疗', '影迷评《速度与激情10》：场面炸裂 刺激过瘾', '《十年一品温如言》丁禹兮任敏分获金扫帚奖', '朱一龙和日本著名导演是枝裕和在戛纳晚宴聊天', '《隔壁泰山》陷版权纠纷 阿里郎组合发声明维权',